Define `gaussian_filter` & `find_peaks` on `ScalarField` & `VectorField`
========================================================================

In [ ]:
import os
import sys
import pyphare

In [ ]:
home = os.environ['HOME']
work_path = os.path.join(home, 'far/farMe/uFunc')
src_path = os.path.join(home, 'far/PHARE')

In [ ]:
# sys.path.append(os.path.join(src_path, "pyphare"))

In [ ]:
import subprocess
import pyphare
import matplotlib.pyplot as plt
from pyphare.pharesee.run import Run
from pyphare.core.operators import dot, cross, sqrt, modulus, grad
from pyphare.core.ufuncs import gFilt, gF, peakIds
import numpy as np

In [ ]:
Te = 0.2

In [ ]:
run_path = os.path.join(work_path, 'wp_{0}'.format(Te))

In [ ]:
if os.path.isdir(run_path):
    files = os.listdir(run_path)
else:
    files = []

In [ ]:
if 'ions_charge_density.h5' not in files :
    os.chdir(work_path)
    subprocess.call(['/usr/bin/python3', work_path+'/wp.py', str(Te)])
    # subprocess.call(['mpirun', '-n', '4', '/usr/bin/python3', work_path+'/wp.py', str(Te)])
    # essayer avec "run" au lieu de "call", qui accepte a priori env=os.environ

In [ ]:
run  = Run(run_path)
time = 60.0

# Operation on `ScalarField`

In [ ]:
N = run.GetNi(time)
type(N)

In [ ]:
fig, ax = plt.subplots(figsize=(6,2))

N.plot(qty='value', ax=ax, ls='solid', lw=0.8, color='tab:blue', ylabel='charge density')

In [ ]:
# n = gFilt(N, sigma=6)

In [ ]:
n = gF(N, sigma=6)

In [ ]:
pks_plus = peakIds(n, height=1.05)
pks_minus = peakIds(-n, names=['value',], height=-0.95)

In [ ]:
fig, ax = plt.subplots(figsize=(6,2))

N.plot(qty='value', ax=ax, ls='solid', lw=1.0, color='tab:blue', ylabel='charge density')
n.plot(qty='value', ax=ax, ls='solid', lw=2.0, color='tab:orange', ylabel='charge density')
for p in pks_plus:
    ax.axvline(x=p, color='black', linestyle='dashed')
for p in pks_minus:
    ax.axvline(x=p, color='black', linestyle='dotted')

# Operation on `VectorField`

In [ ]:
V = run.GetVi(time)

In [ ]:
fig, ax = plt.subplots(figsize=(6,2))

V.plot(qty='x', ax=ax, ls='solid', lw=1.0, color='tab:blue', ylabel='X-velocity')

In [ ]:
# v = gFilt(V, sigma=6)

In [ ]:
v = gF(V, sigma=6)

In [ ]:
fig, ax = plt.subplots(figsize=(6,2))

V.plot(qty='x', ax=ax, ls='solid', lw=1.0, color='tab:blue', ylabel='X-velocity')
v.plot(qty='x', ax=ax, ls='solid', lw=2.0, color='tab:orange', ylabel='X-velocity')

In [ ]:
pks = peakIds(v, height=0.02)

In [ ]:
fig, ax = plt.subplots(figsize=(6,2))

V.plot(qty='x', ax=ax, ls='solid', lw=1.0, color='tab:blue', ylabel='X-velocity')
v.plot(qty='x', ax=ax, ls='solid', lw=2.0, color='tab:orange', ylabel='X-velocity')
for p in pks:
    ax.axvline(x=p, color='black', linestyle='dotted')

# Get an interpolator on a `ScalarField`

In [ ]:
inter = N.interpol(time)

In [ ]:
fn, x_ = inter['value']
x = np.asarray(x_[0])
x = np.arange(0, 150)

In [ ]:
fig, ax = plt.subplots(figsize=(6,2))

ax.plot(x, fn(x))
ax.set_xlabel('x')
ax.set_ylabel('charge density')

# Get an interpolator on a `VectorField`

In [ ]:
inter = V.interpol(time)

In [ ]:
fvx, x_ = inter['x']
x = np.asarray(x_[0])

In [ ]:
fig, ax = plt.subplots(figsize=(6,2))

ax.plot(x, fvx(x))
ax.set_xlabel('x')
ax.set_ylabel('X-velocity')

In [ ]:
for ilvl, lvl in N.levels(time).items():
    for patch in lvl.patches:
        names = list(patch.patch_datas.keys())
        print('names      : {}'.format(names))
        print("id         : {}".format(patch.id))
        print("box        : {}".format(patch.patch_datas[list(patch.patch_datas.keys())[0]].box))
        print(dir(patch.patch_datas[list(patch.patch_datas.keys())[0]]))
        for name in names:
            pdata = patch.patch_datas[name]
            # print("name       : {}".format(name))
            print("size [{}]  : {}   -   {}".format(name, pdata.size, pdata.centerings))
        print(" --- ")
        break